# Aula 5 - Análise de Dados

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from collections import Counter

In [ ]:
dados = pd.read_json('../bases/viajar.ndjson', lines=True)
dados = pd.json_normalize(dados['data'])

colunas_desejadas = ['id', 'desc', 'challenges', 'createTime', 'video.duration', 'video.cover', 'author.id',
                    'author.nickname', 'author.uniqueId','author.verified', 'music.title', 'authorStats.followingCount',
                    'authorStats.followerCount', 'authorStats.heartCount', 'authorStats.videoCount',
                    'authorStats.diggCount', 'authorStats.heart', 'statsV2.collectCount', 'statsV2.commentCount',
                    'statsV2.diggCount', 'statsV2.playCount', 'statsV2.shareCount']

dados = dados[colunas_desejadas]
dados['createTime'] = pd.to_datetime(dados['createTime'], unit='s')

dados['statsV2.diggCount'] = dados['statsV2.diggCount'].astype(float)
dados['statsV2.playCount'] = dados['statsV2.playCount'].astype(float)
dados['statsV2.shareCount'] = dados['statsV2.shareCount'].astype(float)
dados['statsV2.commentCount'] = dados['statsV2.commentCount'].astype(float)
dados['statsV2.collectCount'] = dados['statsV2.collectCount'].astype(float)

def preenche_url(item):
    return f"https://www.tiktok.com/@{item['author.uniqueId']}/video/{item['id']}"

dados['url'] = dados.apply(preenche_url, axis=1)

### Análise Exploratória 

In [ ]:
dados.info()

In [ ]:
dados.describe()

In [ ]:
n_linhas, n_colunas = dados.shape
print(f'A base tem {n_linhas} linhas e {n_colunas} colunas')

In [ ]:
plt.figure(figsize=(8, 4))
sns.set_style('darkgrid')
sns.histplot(data=dados, x='statsV2.diggCount', bins=100, kde=True, color='green')
plt.title('Histograma de likes')
plt.xlabel('Número de likes')
plt.ylabel('Número de vídeos')


In [ ]:
plt.figure(figsize=(8, 4))
sns.set_style('darkgrid')
sns.histplot(data=dados, x='statsV2.commentCount', bins=50, kde=True, color='red')
plt.title('Histograma de Comentários')
plt.xlabel('Número de comentários')
plt.ylabel('Número de vídeos')

In [ ]:
plt.figure(figsize=(8, 4))
sns.set_style('darkgrid')
sns.histplot(data=dados, x='statsV2.shareCount', bins=30, kde=True, color='blue')
plt.title('Histograma de Comentários')
plt.xlabel('Número de comentários')
plt.ylabel('Número de vídeos')

In [ ]:
plt.figure(figsize=(8, 4))
sns.set_style('darkgrid')
sns.histplot(data=dados, x='video.duration', bins=30, kde=True, color='purple')
plt.title('Histograma de Duração dos Vídeos')
plt.xlabel('Duração em segundos')
plt.ylabel('Número de vídeos')

In [ ]:
plt.figure(figsize=(8, 4))
sns.scatterplot(data=dados, x='statsV2.shareCount', y='statsV2.diggCount', color='purple')
plt.title('Dispersão de likes e compartilhamentos')
plt.xlabel('Número de compartilhamentos')
plt.ylabel('Número de likes')
plt.show()

In [ ]:
plt.figure(figsize=(8, 4))
sns.scatterplot(data=dados, x='statsV2.shareCount', y='statsV2.diggCount', color='purple')
plt.title('Dispersão de likes e compartilhamentos')
plt.xlabel('Número de compartilhamentos')
plt.ylabel('Número de likes')
plt.xlim(0, 80_000)
plt.ylim(0, 400_000)

In [ ]:
plt.figure(figsize=(8, 4))
sns.regplot(data=dados, x='statsV2.shareCount', y='statsV2.diggCount', color='purple')
plt.title('Dispersão de likes e compartilhamentos')
plt.xlabel('Número de compartilhamentos')
plt.ylabel('Número de likes')

In [ ]:
plt.figure(figsize=(8, 4))
sns.scatterplot(data=dados, x='statsV2.playCount', y='statsV2.commentCount', color='purple')
plt.show()

In [ ]:
sns.pairplot(data=dados, vars=['statsV2.playCount', 'statsV2.diggCount', 'statsV2.shareCount', 'statsV2.commentCount'])


In [ ]:
dados.groupby('author.uniqueId').agg({'statsV2.diggCount': 'mean',
                                        'statsV2.playCount': 'mean',
                                        'statsV2.shareCount': 'mean',
                                        'statsV2.commentCount': 'mean'}).sort_values(by='statsV2.diggCount', ascending=False).head(20)

## Se eu fosse a VoePass, quais influencers da lista eu usaria para fazer uma campanha de marketing em prol da empresa?

#### Tempo

In [ ]:
dados['month'] = dados['createTime'].dt.to_period('M')
plt.figure(figsize=(20, 4))
sns.countplot(x=dados.sort_values(by='month', ascending=True)['month'], color='purple')
plt.title('Número de vídeos por mês')
plt.xticks(rotation=45)
plt.show()

### Hashtags

In [ ]:
def get_hashtags(lista):
  resultado = []
  try:
    for item in lista:
      resultado.append(item['title'])
    return resultado
  except:
    return resultado
  
dados['hashtags'] = dados['challenges'].apply(get_hashtags)

pd.DataFrame(Counter(dados['hashtags'].sum()).most_common(30))

In [ ]:
wordcloud = WordCloud(width=800, height=400, max_font_size=150, max_words=100, background_color='white').generate(' '.join(dados['hashtags'].sum()))
plt.figure(figsize=(10, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()